# FEM2D Validation Suite

Single-notebook validation for rectangular plate analyses (static, transient, and natural vibration) against the Fortran reference solver.


In [1]:
from __future__ import annotations

from pathlib import Path
import re
import sys
import subprocess
import math
import numpy as np

def _find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for c in candidates:
        if (c / "fortran_reference" / "fem2d.exe").exists() and (c / "Python" / "src").exists():
            return c.resolve()
    raise RuntimeError("Could not locate repository root.")

REPO_ROOT = _find_repo_root()
PYTHON_DIR = REPO_ROOT / "Python"
FORTRAN_EXE = REPO_ROOT / "fortran_reference" / "fem2d.exe"
INPUTS_DIR = REPO_ROOT / "inputs"
OUTPUTS_DIR = REPO_ROOT / "outputs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

if str(PYTHON_DIR) not in sys.path:
    sys.path.insert(0, str(PYTHON_DIR))

from src.driver import solve_from_inp
from plate_modal_python import solve_modal_from_inp

print(f"Repo root : {REPO_ROOT}")
print(f"Fortran   : {FORTRAN_EXE}")


Repo root : C:\Users\arpan\OneDrive\Desktop\FEM2D
Fortran   : C:\Users\arpan\OneDrive\Desktop\FEM2D\fortran_reference\fem2d.exe


In [2]:
def generate_big_mesh_inputs(nx: int = 20, ny: int = 12) -> dict[str, Path]:
    x0, y0 = 0.0, 0.0
    dx = [1.0 / nx] * nx
    dy = [1.0 / ny] * ny
    nxx1 = nx + 1
    nyy1 = ny + 1

    boundary_nodes: list[int] = []
    for j in range(nyy1):
        for i in range(nxx1):
            node = j * nxx1 + i + 1
            if i == 0 or i == nx or j == 0 or j == ny:
                boundary_nodes.append(node)

    pairs: list[int] = []
    for node in boundary_nodes:
        for dof in (1, 2, 3):
            pairs.extend([node, dof])

    nspv = len(boundary_nodes) * 3
    material = "3.0E7 3.0E7 0.3 1.153846153846E7 1.153846153846E7 1.153846153846E7 0.01"

    def fmt(vals: list[float]) -> str:
        return " ".join(f"{v:.12f}" for v in vals)

    def write(name: str, lines: list[str]) -> Path:
        p = INPUTS_DIR / name
        p.write_text("\n".join(lines) + "\n", encoding="utf-8")
        return p

    static_inp = write(
        "rect_plate_static_big.inp",
        [
            "FSDT rectangular plate static big mesh",
            "3 0 0 0",
            "1 4 1 0",
            f"{nx} {ny}",
            "0.0 " + fmt(dx),
            "0.0 " + fmt(dy),
            str(nspv),
            " ".join(str(v) for v in pairs),
            " ".join("0.0" for _ in range(nspv)),
            "0",
            material,
            "1.0 0.0 0.0",
        ],
    )

    transient_inp = write(
        "rect_plate_transient_big.inp",
        [
            "FSDT rectangular plate transient big mesh",
            "3 0 2 0",
            "1 4 1 0",
            f"{nx} {ny}",
            "0.0 " + fmt(dx),
            "0.0 " + fmt(dy),
            str(nspv),
            " ".join(str(v) for v in pairs),
            " ".join("0.0" for _ in range(nspv)),
            "0",
            material,
            "1.0 0.0 0.0",
            "1.0 0.0 0.0",
            "40 41 5 0",
            "0.02 0.25 0.5 1.0E-8",
        ],
    )

    eigen_inp = write(
        "rect_plate_eigen_big.inp",
        [
            "FSDT rectangular plate eigen big mesh",
            "3 0 2 1",
            "10 0",
            "1 4 1 0",
            f"{nx} {ny}",
            "0.0 " + fmt(dx),
            "0.0 " + fmt(dy),
            str(nspv),
            " ".join(str(v) for v in pairs),
            material,
            "1.0 0.0 0.0",
        ],
    )

    print(f"Generated inputs with nx={nx}, ny={ny}, nodes={(nx+1)*(ny+1)}, nspv={nspv}")
    return {"static": static_inp, "transient": transient_inp, "eigen": eigen_inp}

def run_fortran(inp: Path, out_log: Path) -> Path:
    with open(inp, "r", encoding="utf-8") as fin, open(out_log, "w", encoding="utf-8") as fout:
        cp = subprocess.run([str(FORTRAN_EXE)], stdin=fin, stdout=fout, stderr=subprocess.STDOUT, text=True, timeout=300)
    if cp.returncode != 0:
        raise RuntimeError(f"Fortran failed for {inp}")
    return out_log

def run_python_static_or_transient(inp: Path, out_txt: Path) -> Path:
    solve_from_inp(str(inp), output_txt_path=str(out_txt))
    return out_txt

def run_python_modal(inp: Path, out_txt: Path) -> Path:
    solve_modal_from_inp(str(inp), output_file=str(out_txt))
    return out_txt

_row_re = re.compile(r"^\s*(\d+)\s+([+-]?\d+\.\d+E[+-]\d+)\s+([+-]?\d+\.\d+E[+-]\d+)(.*)$")
_num_re = re.compile(r"[+-]?\d+\.\d+E[+-]\d+")

def parse_last_nodal_block(path: Path, ndf: int) -> np.ndarray:
    lines = path.read_text(encoding="utf-8", errors="ignore").splitlines()
    blocks: list[np.ndarray] = []
    i = 0
    while i < len(lines):
        if re.search(r"\bNode\b", lines[i]) and ("x-coord" in lines[i]):
            data = []
            j = i + 1
            while j < len(lines):
                m = _row_re.match(lines[j])
                if not m:
                    if data:
                        break
                    j += 1
                    continue
                nums = [float(x) for x in _num_re.findall(m.group(4))]
                if len(nums) >= ndf:
                    vals = nums[:ndf]
                    data.append((int(m.group(1)), *vals))
                j += 1
            if data:
                arr = np.array(data, dtype=float)
                blocks.append(arr)
            i = j
        else:
            i += 1

    if not blocks:
        raise ValueError(f"No nodal solution block found in {path}")
    last = blocks[-1]
    last = last[np.argsort(last[:, 0])]
    return last[:, 1:1+ndf]

_ft_modal_re = re.compile(
    r"Eigenvalue\(\s*(\d+)\s*\)\s*=\s*([+-]?\d*\.?\d+(?:[EDed][+-]?\d+)?)\s*Frequency\s*=\s*([+-]?\d*\.?\d+(?:[EDed][+-]?\d+)?)"
)

def parse_fortran_modal(log_path: Path) -> list[tuple[int, float, float]]:
    out = []
    txt = log_path.read_text(encoding="utf-8", errors="ignore")
    for m in _ft_modal_re.finditer(txt):
        mode = int(m.group(1))
        lam = float(m.group(2).replace("D", "E").replace("d", "E"))
        frq = float(m.group(3).replace("D", "E").replace("d", "E"))
        out.append((mode, lam, frq))
    return out

def parse_python_modal(txt_path: Path) -> list[tuple[int, float, float]]:
    out = []
    for line in txt_path.read_text(encoding="utf-8", errors="ignore").splitlines():
        parts = line.split()
        if len(parts) >= 3 and parts[0].isdigit():
            out.append((int(parts[0]), float(parts[1]), float(parts[2])))
    return out

def compare_field(fortran_path: Path, python_path: Path, ndf: int = 3) -> dict[str, float]:
    ft = parse_last_nodal_block(fortran_path, ndf)
    py = parse_last_nodal_block(python_path, ndf)
    if ft.shape != py.shape:
        raise ValueError(f"Shape mismatch: Fortran {ft.shape}, Python {py.shape}")
    diff = py - ft
    ft_norm = float(np.linalg.norm(ft.ravel()))
    rel_l2 = float(np.linalg.norm(diff.ravel()) / (ft_norm if ft_norm > 0 else 1.0))
    max_abs = float(np.max(np.abs(diff)))
    return {"rel_l2": rel_l2, "max_abs": max_abs}

def compare_modal(fortran_path: Path, python_path: Path) -> dict[str, float]:
    ft = sorted(parse_fortran_modal(fortran_path), key=lambda x: x[1])
    py = sorted(parse_python_modal(python_path), key=lambda x: x[1])
    n = min(len(ft), len(py))
    if n == 0:
        raise ValueError("No modal data found")
    eig_err = []
    frq_err = []
    for i in range(n):
        lam_ft, lam_py = ft[i][1], py[i][1]
        f_ft, f_py = ft[i][2], py[i][2]
        eig_err.append(abs(lam_py - lam_ft) / (abs(lam_ft) if lam_ft != 0 else 1.0))
        frq_err.append(abs(f_py - f_ft) / (abs(f_ft) if f_ft != 0 else 1.0))
    return {
        "modes_compared": float(n),
        "eig_rel_mean": float(np.mean(eig_err)),
        "eig_rel_max": float(np.max(eig_err)),
        "frq_rel_mean": float(np.mean(frq_err)),
        "frq_rel_max": float(np.max(frq_err)),
    }


In [3]:
inputs = generate_big_mesh_inputs(nx=20, ny=12)

results = []

# Static
static_inp = inputs["static"]
static_ft = OUTPUTS_DIR / f"{static_inp.stem}_fortran.log"
static_py = OUTPUTS_DIR / f"{static_inp.stem}_python.txt"
run_fortran(static_inp, static_ft)
run_python_static_or_transient(static_inp, static_py)
results.append(("static", compare_field(static_ft, static_py, ndf=3)))

# Transient
trans_inp = inputs["transient"]
trans_ft = OUTPUTS_DIR / f"{trans_inp.stem}_fortran.log"
trans_py = OUTPUTS_DIR / f"{trans_inp.stem}_python.txt"
run_fortran(trans_inp, trans_ft)
run_python_static_or_transient(trans_inp, trans_py)
results.append(("transient", compare_field(trans_ft, trans_py, ndf=3)))

# Eigen / natural vibration
eig_inp = inputs["eigen"]
eig_ft = OUTPUTS_DIR / f"{eig_inp.stem}_fortran_modal.log"
eig_py = OUTPUTS_DIR / f"{eig_inp.stem}_python_modal.txt"
run_fortran(eig_inp, eig_ft)
run_python_modal(eig_inp, eig_py)
results.append(("eigen", compare_modal(eig_ft, eig_py)))

for name, metrics in results:
    print(f"\n[{name.upper()}]")
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"  {k:>14}: {v:.6e}")
        else:
            print(f"  {k:>14}: {v}")

print("\nOutput files written to:", OUTPUTS_DIR)


Generated inputs with nx=20, ny=12, nodes=273, nspv=192


Modal solve complete: FSDT rectangular plate eigen big mesh
Input : C:\Users\arpan\OneDrive\Desktop\FEM2D\inputs\rect_plate_eigen_big.inp
Output: C:\Users\arpan\OneDrive\Desktop\FEM2D\outputs\rect_plate_eigen_big_python_modal.txt
Mode 1 eigenvalue: 2.53737999e+12
Mode 1 frequency : 1.59291556e+06

[STATIC]
          rel_l2: 1.493830e-05
         max_abs: 5.000000e-08

[TRANSIENT]
          rel_l2: 1.380235e-05
         max_abs: 5.000000e-04

[EIGEN]
  modes_compared: 1.000000e+01
    eig_rel_mean: 1.130180e-04
     eig_rel_max: 3.547659e-04
    frq_rel_mean: 6.189482e-05
     frq_rel_max: 2.016481e-04

Output files written to: C:\Users\arpan\OneDrive\Desktop\FEM2D\outputs
